In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import polars as pl
import numpy as np

import matplotlib.pyplot as plt

import proyeccion_rdr.features as features

plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

# Asumiendo que es una ocupacion del 80% de camas
INDICE_OCUPACION_CAMAS = 1.25

# Agrega las columnas de poblacion de interes
COLUMNAS_POBLACION_INTERES = {str(i): i for i in range(2017, 2036)}

2025-03-31 15:32:12.354 | INFO     | proyeccion_rdr.config:<module>:11 - PROJ_ROOT path is: C:\Users\jarojas\Desktop\Javier\SSMN\EPH - RDR\2.0-proyeccion-rdr


# Proyección de casos INT al 2035

In [3]:
# Lee los casos de todos los macroprocesos
RUTA_ARCHIVOS = "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx"
_, casos_macroproceso = features.leer_casos_macroprocesos(RUTA_ARCHIVOS)

# Obtiene los casos hospitalizados
casos_hospitalizados = casos_macroproceso.query("tipo_paciente == 'hospitalizados'")

In [4]:
# Los diagnosticos que tienen un porcentaje de hosp. mayor a 0
diagnosticos_relevantes = list(casos_hospitalizados.index)

# Ademas, los diagnosticos que estan agrupados dentro de los que aportan a AC
# Solo deja los diagnosticos que esten agrupados
diagnosticos_a_reasignar = casos_hospitalizados.dropna(subset="Diagnosticos Contenidos")

# Separa la lista de diagnosticos en filas
diagnosticos_agrupados = diagnosticos_a_reasignar.explode("Diagnosticos Contenidos")
diagnosticos_agrupados = list(diagnosticos_agrupados["Diagnosticos Contenidos"].unique())

# Suma los diagnosticos relevantes y los agrupados
diagnosticos_a_buscar_en_egresos = sorted(diagnosticos_relevantes + diagnosticos_agrupados)

## Obtención de casos atendidos entre 2017 y 2020 para diagnósticos relevantes.

Estos se obtendrán del análisis de egresos hospitalarios a nivel país DEIS.

In [5]:
# Lee los egresos del Torax
egresos_rdr = pl.read_csv("../data/raw/4_egresos_deis/egresos_procesados_109101.csv")

En primer lugar, se filtrarán ambas bases de datos para solamente tener la información de los diagnósticos más relevantes para el INT. Luego de esto, se calculará la cantidad de egresos, dias de estada y pacientes para cada uno de los diagnósticos. Estos insumos serán utilizados para estimar la demanda.


In [6]:
# Filtra solamente los diagnosticos mas relevantes del Torax
egresos_mas_relevantes_rdr = egresos_rdr.filter(
    (pl.col("DIAG1").is_in(diagnosticos_a_buscar_en_egresos))
)

## Reasginación de diagnósticos y agrupación

In [7]:
for row in diagnosticos_a_reasignar.itertuples():
    diagnostico_nuevo = row[0]
    diagnosticos_antiguos = row[1]

    print(f"Cambiando {diagnosticos_antiguos} a {diagnostico_nuevo}")
    egresos_mas_relevantes_rdr = egresos_mas_relevantes_rdr.with_columns(
        pl.col("DIAG1").replace(diagnosticos_antiguos, diagnostico_nuevo)
    )

Cambiando ['A099'] a A090
Cambiando ['J208', 'J209', 'J210', 'J218'] a J205


In [8]:
# Obtiene resumen de metricas para el Torax
metricas_diags_relevantes_rdr = (
    features.calcular_resumen_metricas_desagregadas_y_agrupadas_en_anios(
        egresos_mas_relevantes_rdr, 2019, 2023
    )
)

In [9]:
# Define las columnas que se quieran corregir
columna_dias_estada = ("agrupado_entre_2019_2023", "dias_estada_promedio")

## Estimación de casos Hospitalizados utilizando % de Hospitalización del INT

En este caso, se utilizará el % de hospitalización del INT de los pacientes ambulatorios.

In [10]:
# Se obtiene la cantidad de egresos que debiese tener el INT, asumiendo que trabajara con la misma
# eficiencia
egresos_estimados_rdr = casos_hospitalizados[COLUMNAS_POBLACION_INTERES.keys()].mul(
    metricas_diags_relevantes_rdr[("agrupado_entre_2019_2023", "egresos_por_paciente")], axis=0
)

# Se obtiene la cantidad de dias de estada que debiese el INT, asumiendo que trabajara con la misma
# eficiencia
dias_estada_estimados_rdr = egresos_estimados_rdr.mul(
    metricas_diags_relevantes_rdr[("agrupado_entre_2019_2023", "dias_estada_promedio")], axis=0
)

# Estima la cantidad de camas necesaarias por diagnostico
camas_estimadas_rdr = (dias_estada_estimados_rdr / 365.25) * INDICE_OCUPACION_CAMAS

# Calcula las camas totales necesarias
camas_totales_rdr = camas_estimadas_rdr.sum()

In [11]:
# Se quiere tener un 60% de UPC (1:2, UCI:UTI, y 40% de Medias)
FRACCION_UCI = 0.21
FRACCION_UTI = 0.42
FRACCION_MEDIAS = 0.37

# Agrega las camas necesarias para trasplantes a las del 2035
camas_al_2035 = (camas_totales_rdr["2035"])
# Obtiene la cantidad de camas UCI, UTI y Medias sin redondear
camas_uci = (camas_al_2035 * FRACCION_UCI)
camas_uti = (camas_al_2035 * FRACCION_UTI)
camas_medias = (camas_al_2035 * FRACCION_MEDIAS)

# Obtiene la cantidad de camas UCI, UTI y Medias redondeadas
camas_uci_redondeadas = round(camas_al_2035 * FRACCION_UCI)
camas_uti_redondeadas = round(camas_al_2035 * FRACCION_UTI)
camas_medias_redondeadas = round(camas_al_2035 * FRACCION_MEDIAS)
camas_al_2035_redondeadas = round(camas_al_2035)

In [12]:
print(f"> Camas 2035: {camas_al_2035:.2f}")
# print(f"> UCI: {camas_uci:.2f} camas")
# print(f"> UTI: {camas_uti:.2f} camas")
# print(f"> Medias: {camas_medias:.2f} camas")
print()
# print(f"> UCI Redondeadas: {camas_uci_redondeadas} camas")
# print(f"> UTI Redondeadas: {camas_uti_redondeadas} camas")
# print(f"> Medias Redondeadas: {camas_medias_redondeadas} camas")
print(f"> Total Redondeadas: {camas_al_2035_redondeadas} camas")

> Camas 2035: 137.53

> Total Redondeadas: 138 camas


## Obtener resumen para MINSAL

In [13]:
# Obtiene porcentaje de hospitalizados
porcentaje_hospitalizados = pd.read_excel(
    "../data/raw/3_incidencias_y_porcentajes_marcoprocesos/incidencias_y_prevalencias_RDR.xlsx"
)

porcentaje_hospitalizados["Diagnostico"] = (
    porcentaje_hospitalizados["Diagnostico"].str.split(" - ").str[0]
)

# Filtra los porcentajes de hospitalizados
porcentaje_hospitalizados = porcentaje_hospitalizados.set_index("Diagnostico")
porcentaje_hospitalizados = porcentaje_hospitalizados["Porcentaje Pacientes Hospitalizados"]
porcentaje_hospitalizados = porcentaje_hospitalizados.dropna()
porcentaje_hospitalizados = porcentaje_hospitalizados[porcentaje_hospitalizados > 0]

In [14]:
# Obtiene resumen para MINSAL
resumen_hosp = pd.DataFrame(
    {
        "porcentaje_hospitalizado": porcentaje_hospitalizados,
        "casos_hospitalizados_2035": casos_hospitalizados["2035"],
        "egresos_por_paciente_agrupado_2019_2023": metricas_diags_relevantes_rdr[
            ("agrupado_entre_2019_2023", "egresos_por_paciente")
        ],
        "egresos_2035": egresos_estimados_rdr["2035"],
        "dias_estada_promedio_agrupado_2019_2023": metricas_diags_relevantes_rdr[
            ("agrupado_entre_2019_2023", "dias_estada_promedio")
        ],
        "dias_estada_totales_2035": dias_estada_estimados_rdr["2035"],
        "camas_totales_2035": camas_estimadas_rdr["2035"],
    }
)

Finalmente, une el resumen de casos teoricos desde el cuadernillo 1.0 con el resumen de este cuadernillo. Esto obtiene la tabla final para MINSAL.

In [16]:
resumen_casos_teoricos = pd.read_excel(
    "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx", sheet_name="resumen_total_RDR"
)

resumen_casos_teoricos["Diagnostico_codigo"] = (
    resumen_casos_teoricos["Diagnostico"].str.split(" - ").str[0]
)
resumen_casos_teoricos = resumen_casos_teoricos.set_index("Diagnostico_codigo")

resumen_minsal = resumen_casos_teoricos.merge(
    resumen_hosp, how="right", left_index=True, right_index=True
)

resumen_minsal = resumen_minsal.reset_index(drop=True).set_index("Diagnostico")

## Guardar archivos

In [20]:
archivos_a_guardar = {
    "resumen_MINSAL": resumen_minsal,
    "metricas_relevantes_RDR": metricas_diags_relevantes_rdr,
    "casos_hospitalizados_RDR": casos_hospitalizados,
    "egresos_estimados_RDR": egresos_estimados_rdr,
    "dias_estada_estimados_RDR": dias_estada_estimados_rdr,
    "camas_estimadas_desglosadas_RDR": camas_estimadas_rdr,
    "camas_totales_RDR": camas_totales_rdr,
    "resumen_total_hosp_RDR": resumen_hosp,
}

with pd.ExcelWriter("../data/interim/1.0_estimacion_atencion_cerrada_RDR.xlsx") as file:
    for nombre_hoja, df_a_guardar in archivos_a_guardar.items():
        print(f"Guardando {nombre_hoja}")
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)

Guardando resumen_MINSAL
Guardando metricas_relevantes_RDR
Guardando casos_hospitalizados_RDR
Guardando egresos_estimados_RDR
Guardando dias_estada_estimados_RDR
Guardando camas_estimadas_desglosadas_RDR
Guardando camas_totales_RDR
Guardando resumen_total_hosp_RDR
